In [1]:
!pip install torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
!pip install matplotlib

In [3]:
!pip install opencv-contrib-python

In [4]:
!pip install imutils

In [5]:
!pip install scikit-learn

In [6]:
!pip install tqdm

In [22]:
# Mount Google Drive (if not already mounted)
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [25]:
import zipfile
import os

# Correct path with the right filename (MSFD.zip not MFSD.zip)
zip_path = '/content/drive/MyDrive/Colab Notebooks/MSFD.zip'

# Create the extraction directory
os.makedirs('MFSD', exist_ok=True)

# Extract the zip file
try:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        print("Extracting files... (this may take a while as the file is ~2.25GB)")
        zip_ref.extractall('MFSD')
    print('Extraction complete!')
except FileNotFoundError:
    print(f"File not found: {zip_path}")
except zipfile.BadZipFile:
    print("The file is not a valid zip file.")

Extracting files... (this may take a while as the file is ~2.25GB)
Extraction complete!


In [28]:
# Print the current content of your config.py file
!cat /content/imageSearch/config.py

# define the number of channels in the input, number of classes,
# and number of levels in the U-Net model
NUM_CHANNELS = 1
NUM_CLASSES = 1
NUM_LEVELS = 3
# initialize learning rate, number of epochs to train for, and the
# batch size
INIT_LR = 0.001
NUM_EPOCHS = 40
BATCH_SIZE = 64
# define the input image dimensions
INPUT_IMAGE_WIDTH = 128
INPUT_IMAGE_HEIGHT = 128
# define threshold to filter weak predictions
THRESHOLD = 0.5
# define the path to the base output directory
BASE_OUTPUT = "output"
# define the path to the output serialized model, model training
# plot, and testing image paths

import os

MODEL_PATH = os.path.join(BASE_OUTPUT, "unet_tgs_salt.pth")
PLOT_PATH = os.path.sep.join([BASE_OUTPUT, "plot.png"])
TEST_PATHS = os.path.sep.join([BASE_OUTPUT, "test_paths.txt"])

# Edit the config.py file to add the missing path variables
with open('/content/imageSearch/config.py', 'r') as file:
    content = file.read()

# Add the necessary path variables if they don't exist
required_pa

In [29]:
# Create a new config.py file with all the necessary variables
with open('/content/imageSearch/config.py', 'w') as file:
    file.write("""
import os

# Define the path to the dataset
DATASET_PATH = "MFSD"

# Define the path to the images and masks dataset
IMAGE_DATASET_PATH = os.path.join(DATASET_PATH, "images")
MASK_DATASET_PATH = os.path.join(DATASET_PATH, "masks")

# Define the test split
TEST_SPLIT = 0.15

# Determine the device to be used for training and evaluation
import torch
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Determine if we will be pinning memory during data loading
PIN_MEMORY = True if DEVICE == "cuda" else False

# Define the number of channels in the input, number of classes,
# and number of levels in the U-Net model
NUM_CHANNELS = 1
NUM_CLASSES = 1
NUM_LEVELS = 3

# Initialize learning rate, number of epochs to train for, and the
# batch size
INIT_LR = 0.001
NUM_EPOCHS = 40
BATCH_SIZE = 64

# Define the input image dimensions
INPUT_IMAGE_WIDTH = 128
INPUT_IMAGE_HEIGHT = 128

# Define threshold to filter weak predictions
THRESHOLD = 0.5

# Define the path to the base output directory
BASE_OUTPUT = "output"

# Define the path to the output serialized model, model training
# plot, and testing image paths
MODEL_PATH = os.path.join(BASE_OUTPUT, "unet_face_mask.pth")
PLOT_PATH = os.path.sep.join([BASE_OUTPUT, "plot.png"])
TEST_PATHS = os.path.sep.join([BASE_OUTPUT, "test_paths.txt"])

# Create the output directory if it doesn't exist
os.makedirs(BASE_OUTPUT, exist_ok=True)
""")

print("Created a new config.py file with all required variables")

Created a new config.py file with all required variables


In [30]:
# Print the import statements from your train.py
!head -20 /content/train.py

# USAGE
# python train.py
# import the necessary packages
from imageSearch.dataset import SegmentationDataset
from imageSearch.model import UNet
from imageSearch import config
from torch.nn import BCEWithLogitsLoss
from torch.optim import Adam
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torchvision import transforms
from imutils import paths
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
import time
import os

# load the image and mask filepaths in a sorted manner
imagePaths = sorted(list(paths.list_images(config.IMAGE_DATASET_PATH)))


In [31]:
!pip install imutils

In [33]:
# Update the paths in your config file based on your folder structure
with open('/content/imageSearch/config.py', 'r') as file:
    content = file.read()

# Replace the path variables with the correct ones
content = content.replace(
    'DATASET_PATH = "MFSD"',
    'DATASET_PATH = "MFSD/MSFD/1"'  # Point to the "1" folder inside MSFD
)
content = content.replace(
    'IMAGE_DATASET_PATH = os.path.join(DATASET_PATH, "images")',
    'IMAGE_DATASET_PATH = os.path.join(DATASET_PATH, "face_crop")'  # Assuming this is where the input images are
)
content = content.replace(
    'MASK_DATASET_PATH = os.path.join(DATASET_PATH, "masks")',
    'MASK_DATASET_PATH = os.path.join(DATASET_PATH, "face_crop_segmentation")'  # Assuming this is where the masks are
)

# Write the updated content back
with open('/content/imageSearch/config.py', 'w') as file:
    file.write(content)

print("Updated config.py with corrected paths")

Updated config.py with corrected paths


In [34]:
# Count image files in the expected locations
!ls -la MFSD/MSFD/1/face_crop | head -5
!ls -la MFSD/MSFD/1/face_crop_segmentation | head -5

total 605876
drwxr-xr-x 2 root root 274432 Mar 22 18:45 .
drwxr-xr-x 5 root root   4096 Mar 22 18:45 ..
-rw-r--r-- 1 root root  43595 Mar 22 19:19 000000_1.jpg
-rw-r--r-- 1 root root   2390 Mar 22 19:19 000001_1.jpg
total 74408
drwxr-xr-x 2 root root 274432 Mar 22 18:45 .
drwxr-xr-x 5 root root   4096 Mar 22 18:45 ..
-rw-r--r-- 1 root root   5926 Mar 22 19:19 000000_1.jpg
-rw-r--r-- 1 root root   1039 Mar 22 19:19 000001_1.jpg


In [37]:
!python train.py

[INFO] saving testing image paths...
[INFO] found 7975 examples in the training set...
[INFO] found 1408 examples in the test set...
[INFO] training the network...
  0% 0/40 [00:00<?, ?it/s][INFO] EPOCH: 1/40
Train loss: 0.578668, Test loss: 0.5364
  2% 1/40 [00:47<30:50, 47.44s/it][INFO] EPOCH: 2/40
Train loss: 0.519805, Test loss: 0.4964
  5% 2/40 [01:29<27:52, 44.01s/it][INFO] EPOCH: 3/40
Train loss: 0.499576, Test loss: 0.4973
  8% 3/40 [02:10<26:24, 42.83s/it][INFO] EPOCH: 4/40
Train loss: 0.475473, Test loss: 0.4552
 10% 4/40 [02:52<25:27, 42.44s/it][INFO] EPOCH: 5/40
Train loss: 0.457837, Test loss: 0.4430
 12% 5/40 [03:34<24:37, 42.22s/it][INFO] EPOCH: 6/40
Train loss: 0.447996, Test loss: 0.4502
 15% 6/40 [04:15<23:49, 42.06s/it][INFO] EPOCH: 7/40
Train loss: 0.442463, Test loss: 0.4475
 18% 7/40 [04:57<23:02, 41.88s/it][INFO] EPOCH: 8/40
Train loss: 0.435304, Test loss: 0.4252
 20% 8/40 [05:38<22:10, 41.58s/it][INFO] EPOCH: 9/40
Train loss: 0.427701, Test loss: 0.4222
 22% 9/

In [40]:
!ls -la output/unet_face_mask.pth

-rw-r--r-- 1 root root 485082 Mar 22 19:59 output/unet_face_mask.pth


In [42]:
# Update the config.py file to include the DEVICE variable
with open('/content/imageSearch/config.py', 'r') as file:
    content = file.read()

# Check if DEVICE is already defined
if 'DEVICE =' not in content:
    # Add the device configuration
    device_config = """
# Determine the device to be used for training and evaluation
import torch
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Determine if we will be pinning memory during data loading
PIN_MEMORY = True if DEVICE == "cuda" else False
"""

    # Add it after the imports
    if 'import os' in content:
        content = content.replace('import os', 'import os\nimport torch')
        # Add the device config after the path definitions
        if 'MASK_DATASET_PATH =' in content:
            content = content.replace('MASK_DATASET_PATH =', 'MASK_DATASET_PATH =\n\n' + device_config.strip() + '\n\n# MASK_DATASET_PATH =')
        else:
            # If no MASK_DATASET_PATH, add it at the end
            content += '\n' + device_config

    # Write the updated content back
    with open('/content/imageSearch/config.py', 'w') as file:
        file.write(content)

print("Updated config.py with DEVICE variable")

Updated config.py with DEVICE variable


In [44]:
# Create a fresh, error-free config.py file
with open('/content/imageSearch/config.py', 'w') as file:
    file.write("""import os
import torch

# Define the path to the dataset
DATASET_PATH = "MFSD/MSFD/1"

# Define the path to the images and masks dataset
IMAGE_DATASET_PATH = os.path.join(DATASET_PATH, "face_crop")
MASK_DATASET_PATH = os.path.join(DATASET_PATH, "face_crop_segmentation")

# Define the test split
TEST_SPLIT = 0.15

# Determine the device to be used for training and evaluation
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Determine if we will be pinning memory during data loading
PIN_MEMORY = True if DEVICE == "cuda" else False

# Define the number of channels in the input, number of classes,
# and number of levels in the U-Net model
NUM_CHANNELS = 1
NUM_CLASSES = 1
NUM_LEVELS = 3

# Initialize learning rate, number of epochs to train for, and the
# batch size
INIT_LR = 0.001
NUM_EPOCHS = 40
BATCH_SIZE = 64

# Define the input image dimensions
INPUT_IMAGE_WIDTH = 128
INPUT_IMAGE_HEIGHT = 128

# Define threshold to filter weak predictions
THRESHOLD = 0.5

# Define the path to the base output directory
BASE_OUTPUT = "output"

# Define the path to the output serialized model, model training
# plot, and testing image paths
MODEL_PATH = os.path.join(BASE_OUTPUT, "unet_face_mask.pth")
PLOT_PATH = os.path.sep.join([BASE_OUTPUT, "plot.png"])
TEST_PATHS = os.path.sep.join([BASE_OUTPUT, "test_paths.txt"])

# Create the output directory if it doesn't exist
os.makedirs(BASE_OUTPUT, exist_ok=True)
""")

print("Created a fresh config.py file without syntax errors")

Created a fresh config.py file without syntax errors


In [46]:
# Add IoU and Dice score calculations to predict.py
with open('/content/predict.py', 'r') as file:
    content = file.read()

# Add the calculation functions
metrics_code = """
# Calculate IoU (Intersection over Union)
def calculate_iou(pred_mask, gt_mask):
    # Convert masks to binary format
    pred_mask = (pred_mask > 0).astype(np.uint8)
    gt_mask = (gt_mask > 0).astype(np.uint8)

    # Calculate intersection and union
    intersection = np.logical_and(pred_mask, gt_mask).sum()
    union = np.logical_or(pred_mask, gt_mask).sum()

    # Calculate IoU
    iou = intersection / union if union > 0 else 0
    return iou

# Calculate Dice coefficient
def calculate_dice(pred_mask, gt_mask):
    # Convert masks to binary format
    pred_mask = (pred_mask > 0).astype(np.uint8)
    gt_mask = (gt_mask > 0).astype(np.uint8)

    # Calculate intersection and sum of areas
    intersection = np.logical_and(pred_mask, gt_mask).sum()
    sum_areas = pred_mask.sum() + gt_mask.sum()

    # Calculate Dice
    dice = (2 * intersection) / sum_areas if sum_areas > 0 else 0
    return dice
"""

# Find a good place to insert the metrics code (after imports)
if 'import os' in content:
    content = content.replace('import os', 'import os\n' + metrics_code)
else:
    # Add it after the numpy import
    content = content.replace('import numpy as np', 'import numpy as np\n' + metrics_code)

# Modify the make_predictions function to calculate and display the metrics
if 'make_predictions' in content and 'prepare_plot' in content:
    # Update prepare_plot to include metrics
    content = content.replace(
        'def prepare_plot(origImage, origMask, predMask):',
        'def prepare_plot(origImage, origMask, predMask, iou, dice):'
    )

    content = content.replace(
        'ax[2].set_title("Predicted Mask")',
        'ax[2].set_title(f"Predicted Mask\\nIoU: {iou:.4f}, Dice: {dice:.4f}")'
    )

    # Update make_predictions to calculate metrics
    metrics_calculation = """
        # Calculate IoU and Dice scores
        iou = calculate_iou(predMask, gtMask)
        dice = calculate_dice(predMask, gtMask)
        print(f"IoU: {iou:.4f}, Dice: {dice:.4f}")

        # prepare a plot for visualization
        prepare_plot(orig, gtMask, predMask, iou, dice)
"""

    # Find where to insert the metrics calculation
    if 'prepare_plot(orig, gtMask, predMask)' in content:
        content = content.replace(
            'prepare_plot(orig, gtMask, predMask)',
            metrics_calculation.strip()
        )

# Write the modified content back
with open('/content/predict.py', 'w') as file:
    file.write(content)

print("Added IoU and Dice score calculations to predict.py")

Added IoU and Dice score calculations to predict.py


In [47]:
# Update the main part of predict.py to calculate average metrics
with open('/content/predict.py', 'r') as file:
    content = file.read()

# Add code to keep track of average metrics
if "for path in imagePaths:" in content:
    # Find the section before the loop
    before_loop = content.split("for path in imagePaths:")[0]
    # Find the section with the loop
    loop_section = "for path in imagePaths:" + content.split("for path in imagePaths:")[1]

    # Add metrics tracking code before the loop
    metrics_tracking = """
# Initialize lists to store metrics
all_ious = []
all_dice_scores = []

"""
    updated_content = before_loop + metrics_tracking + loop_section

    # Update the loop to collect metrics
    updated_content = updated_content.replace(
        "make_predictions(unet, path)",
        "iou, dice = make_predictions(unet, path)\nall_ious.append(iou)\nall_dice_scores.append(dice)"
    )

    # Add code to print average metrics after the loop
    updated_content += """
# Calculate and print average metrics
avg_iou = sum(all_ious) / len(all_ious) if all_ious else 0
avg_dice = sum(all_dice_scores) / len(all_dice_scores) if all_dice_scores else 0
print(f"\\nAverage metrics across {len(all_ious)} images:")
print(f"Average IoU: {avg_iou:.4f}")
print(f"Average Dice score: {avg_dice:.4f}")
"""

    # Write the updated content back
    with open('/content/predict.py', 'w') as file:
        file.write(updated_content)

    print("Updated predict.py to calculate average metrics")

# Also modify make_predictions to return the calculated metrics
with open('/content/predict.py', 'r') as file:
    content = file.read()

if "def make_predictions" in content and "return" not in content:
    # Add return statement to make_predictions
    content = content.replace(
        "prepare_plot(orig, gtMask, predMask, iou, dice)",
        "prepare_plot(orig, gtMask, predMask, iou, dice)\n\t\treturn iou, dice"
    )

    # Write the updated content back
    with open('/content/predict.py', 'w') as file:
        file.write(content)

    print("Modified make_predictions to return metrics")

Updated predict.py to calculate average metrics


In [49]:
# Fix the indentation in predict.py
with open('/content/predict.py', 'r') as file:
    content = file.read()

# Replace all tabs with 4 spaces to standardize indentation
content = content.replace('\t', '    ')

# Write the standardized content back
with open('/content/predict.py', 'w') as file:
    file.write(content)

print("Fixed indentation in predict.py by replacing tabs with spaces")

Fixed indentation in predict.py by replacing tabs with spaces


In [51]:
# Check if test paths file exists and what it contains
!cat output/test_paths.txt | wc -l
!head -5 output/test_paths.txt

1407
MFSD/MSFD/1/face_crop/001812_2.jpg
MFSD/MSFD/1/face_crop/007595_1.jpg
MFSD/MSFD/1/face_crop/007380_2.jpg
MFSD/MSFD/1/face_crop/001801_1.jpg
MFSD/MSFD/1/face_crop/005483_1.jpg


In [52]:
# First, let's check if our modified predict.py is trying to calculate IoU
with open('/content/predict.py', 'r') as file:
    content = file.read()

# Check if the IoU calculation functions are defined
if 'calculate_iou' not in content:
    # Add the IoU and Dice calculation functions
    metrics_code = """
# Calculate IoU (Intersection over Union)
def calculate_iou(pred_mask, gt_mask):
    # Convert masks to binary format
    pred_mask = (pred_mask > 0).astype(np.uint8)
    gt_mask = (gt_mask > 0).astype(np.uint8)

    # Calculate intersection and union
    intersection = np.logical_and(pred_mask, gt_mask).sum()
    union = np.logical_or(pred_mask, gt_mask).sum()

    # Calculate IoU
    iou = intersection / union if union > 0 else 0
    return iou

# Calculate Dice coefficient
def calculate_dice(pred_mask, gt_mask):
    # Convert masks to binary format
    pred_mask = (pred_mask > 0).astype(np.uint8)
    gt_mask = (gt_mask > 0).astype(np.uint8)

    # Calculate intersection and sum of areas
    intersection = np.logical_and(pred_mask, gt_mask).sum()
    sum_areas = pred_mask.sum() + gt_mask.sum()

    # Calculate Dice
    dice = (2 * intersection) / sum_areas if sum_areas > 0 else 0
    return dice
"""
    # Add after imports but before other functions
    if "import cv2" in content:
        content = content.replace("import cv2", "import cv2\n" + metrics_code)
    else:
        content = content.replace("import numpy as np", "import numpy as np\n" + metrics_code)

    # Write back to the file
    with open('/content/predict.py', 'w') as file:
        file.write(content)
    print("Added IoU and Dice calculation functions")

# Now let's modify the make_predictions function to calculate and print IoU
with open('/content/predict.py', 'r') as file:
    content = file.read()

# Find the make_predictions function
make_predictions_start = content.find("def make_predictions")
if make_predictions_start != -1:
    # Check if it already has IoU calculation
    if "calculate_iou" not in content[make_predictions_start:]:
        # Find where to add IoU calculation (right before prepare_plot)
        prepare_plot_pos = content.find("prepare_plot", make_predictions_start)
        if prepare_plot_pos != -1:
            # Insert IoU calculation before prepare_plot
            iou_code = """
        # Calculate IoU and Dice scores
        iou = calculate_iou(predMask, gtMask)
        dice = calculate_dice(predMask, gtMask)
        print(f"File: {filename}, IoU: {iou:.4f}, Dice: {dice:.4f}")

        """
            # Split content to insert our code
            content_before = content[:prepare_plot_pos]
            content_after = content[prepare_plot_pos:]
            # Combine with our IoU code
            content = content_before + iou_code + content_after

            # Write back to the file
            with open('/content/predict.py', 'w') as file:
                file.write(content)
            print("Modified make_predictions to calculate and print IoU")

# Make sure prepare_plot accepts IoU parameters
with open('/content/predict.py', 'r') as file:
    content = file.read()

if "def prepare_plot" in content:
    # Update the prepare_plot function signature and implementation
    prepare_plot_start = content.find("def prepare_plot")
    prepare_plot_end = content.find("def", prepare_plot_start + 1)
    if prepare_plot_end == -1:  # If it's the last function
        prepare_plot_end = len(content)

    # Create updated prepare_plot function
    updated_prepare_plot = """
def prepare_plot(origImage, origMask, predMask, iou=None, dice=None, imagePath=None):
    # initialize our figure
    figure, ax = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))
    # plot the original image, its mask, and the predicted mask
    ax[0].imshow(origImage)
    ax[1].imshow(origMask, cmap='gray')
    ax[2].imshow(predMask, cmap='gray')
    # set the titles of the subplots
    ax[0].set_title("Image")
    ax[1].set_title("Original Mask")
    title = "Predicted Mask"
    if iou is not None and dice is not None:
        title = f"Predicted Mask\\nIoU: {iou:.4f}, Dice: {dice:.4f}"
    ax[2].set_title(title)
    # set the layout of the figure and display it
    figure.tight_layout()
    figure.show()

    # Save the figure if imagePath is provided
    if imagePath:
        os.makedirs("visualizations", exist_ok=True)
        save_path = os.path.join("visualizations", f"segmentation_{os.path.basename(imagePath).split('.')[0]}.png")
        figure.savefig(save_path)
        print(f"Visualization saved to {save_path}")
"""

    # Replace the old function with our updated one
    content = content[:prepare_plot_start] + updated_prepare_plot + content[prepare_plot_end:]

    # Write back to the file
    with open('/content/predict.py', 'w') as file:
        file.write(content)
    print("Updated prepare_plot function to display IoU and Dice scores")

# Make sure the make_predictions function returns IoU and Dice
with open('/content/predict.py', 'r') as file:
    content = file.read()

# Find make_predictions function
make_predictions_start = content.find("def make_predictions")
if make_predictions_start != -1:
    # If there's no return statement, add one
    if "return iou, dice" not in content[make_predictions_start:]:
        # Find the end of the function
        next_def = content.find("def", make_predictions_start + 1)
        if next_def == -1:
            next_def = len(content)

        # Check if there's a prepare_plot call
        prepare_plot_pos = content.rfind("prepare_plot", make_predictions_start, next_def)
        if prepare_plot_pos != -1:
            # Find the end of this line
            line_end = content.find("\n", prepare_plot_pos)
            if line_end != -1:
                # Insert return statement after prepare_plot call
                content = content[:line_end+1] + "        return iou, dice\n" + content[line_end+1:]

                # Write back to the file
                with open('/content/predict.py', 'w') as file:
                    file.write(content)
                print("Added return statement to make_predictions function")

Added IoU and Dice calculation functions
Modified make_predictions to calculate and print IoU
Updated prepare_plot function to display IoU and Dice scores
Added return statement to make_predictions function


In [56]:
!python predict.py

[INFO] loading up test image paths...
Number of test images: 1408
Selected 10 random images for evaluation
[INFO] loading model...
Model loaded successfully
Starting prediction on test images...
Processing image: MFSD/MSFD/1/face_crop/003008_1.jpg
File: 003008_1.jpg, IoU: 0.5651, Dice: 0.7221
Figure(1500x500)
Visualization saved to visualizations/segmentation_003008_1.png
Processing image: MFSD/MSFD/1/face_crop/002164_1.jpg
File: 002164_1.jpg, IoU: 0.7383, Dice: 0.8494
Figure(1500x500)
Visualization saved to visualizations/segmentation_002164_1.png
Processing image: MFSD/MSFD/1/face_crop/002538_1.jpg
File: 002538_1.jpg, IoU: 0.7843, Dice: 0.8791
Figure(1500x500)
Visualization saved to visualizations/segmentation_002538_1.png
Processing image: MFSD/MSFD/1/face_crop/004021_1.jpg
File: 004021_1.jpg, IoU: 0.6370, Dice: 0.7783
Figure(1500x500)
Visualization saved to visualizations/segmentation_004021_1.png
Processing image: MFSD/MSFD/1/face_crop/003042_1.jpg
File: 003042_1.jpg, IoU: 0.7003,